In [1]:
import io, sys
import nbformat as nb
import nbformat.v4.nbbase as nb4

In [2]:
#Are we in a notebook segment?
innb=False
 
#Quick and dirty count of notebooks
c=1
 
#The monolithic notebook is the content ex of the separate notebook content
monolith=nb4.new_notebook()

In [3]:
#Load the original doc in
mynb=nb.read('Metalaxyl Crop Soil 4 Rainfall Patterns.ipynb', nb.NO_CONVERT)

#For each cell in the original doc:
for i in mynb['cells']:
    if (i['cell_type']=='markdown'):
        #See if we can stop a standalone notebook code delimiter
        if ('START NOTEBOOK' in i['source']):
            #At the start of a block, create a new notebook
            innb=True
            test=nb4.new_notebook()
        elif ('END NOTEBOOK' in i['source']):
            #At the end of the block, save the code to a new standalone notebook file
            innb=False
            nb.write(test,'test{}.ipynb'.format(c))
            c=c+1
        elif (innb):
            test.cells.append(nb4.new_markdown_cell(i['source']))
        else:
            monolith.cells.append(nb4.new_markdown_cell(i['source']))
    elif (i['cell_type']=='code'):
        #For the code cells, preserve any output text
        cc=nb4.new_code_cell(i['source'])
        for o in i['outputs']:
            cc['outputs'].append(o)
        #Route the code cell as required...
        if (innb):
            test.cells.append(cc)
        else:
            monolith.cells.append(cc)
 
#Save the monolithic notebook
nb.write(monolith,'monolith.ipynb')
# nb.write(test,'test.ipynb')

In [4]:
#Convert it to markdown
!jupyter nbconvert --to markdown monolith.ipynb
 

[NbConvertApp] Converting notebook monolith.ipynb to markdown
[NbConvertApp] Writing 0 bytes to monolith.md


In [5]:
##On a Mac, I got pandoc via:
#brew install pandoc

#Generate a Microsoft .docx file from the markdown
!pandoc -o monolith.docx -f markdown -t docx monolith.md